In [87]:
### TODO Normalize using groupby('file'); Add seconds column; Add column saying who won
### Potentially normalize x, y positions by map
### WE only dealt with master_grenades_hit, but we could add master_grenades_miss in the future

In [3]:
# file for merging
# round 1-hot encoded
# tick and seconds normalize by match when figured out
# attck_team and vic_team depends on if predicting on dataset or outside dataset because uses only like 15 teams
# vic_side and att_side 1-hot encoded
# normalized hp_dmg and arm_dmg
# is_bomb_planted 1-hot encoded
# bomb_site filled nulls with 0 but we will 1-hot encode it
# need to 1-hot encode hitbox
# need to 1-hot encode 'wp' and 'wp type'
# att_id and vic_id is player id but once again will not really be used because it only works within dataset and not outside it
# att_rank and vic_rank only 1 value, which is 0
# att_pos_x, att_pos_y, vic_pos_x, vic_pos_y should not be normalized or one-hot encoded, so will not be included in X
# bomb site, hitbox, wp, wp type still need to be done

In [4]:
import pandas as pd
import numpy as np

In [5]:
copy_grenades = pd.read_csv('csgo-matchmaking-damage/esea_master_grenades_demos.part1.csv')

In [6]:
copy_meta = pd.read_csv('csgo-matchmaking-damage/esea_meta_demos.part1.csv')

In [11]:
master_grenades = copy_grenades
master_meta = copy_meta

In [16]:
master_grenades.head()

,file,round,seconds,att_team,vic_team,att_id,vic_id,att_side,vic_side,hp_dmg,...,hitbox,nade,att_rank,vic_rank,att_pos_x,att_pos_y,nade_land_x,nade_land_y,vic_pos_x,vic_pos_y
0,esea_match_13770997.dem,1,153.1602,Animal Style,NaN,76561198165334141,NaN,CounterTerrorist,NaN,0,...,NaN,Smoke,0,NaN,-1618.146,-66.00259,-949.8569,-340.3019,NaN,NaN
1,esea_match_13770997.dem,2,184.7945,Hentai Hooligans,Animal Style,76561198037331400,7.656120e+16,Terrorist,CounterTerrorist,70,...,Generic,HE,0,0.0,-1719.904,-2357.64700,-2774.6650,-1603.9430,-2741.25,-1523.163
2,esea_match_13770997.dem,2,186.8617,Animal Style,NaN,76561198055191021,NaN,CounterTerrorist,NaN,0,...,NaN,HE,0,NaN,-1036.352,492.16760,-466.8676,-356.9641,NaN,NaN
3,esea_match_13770997.dem,2,187.1122,Animal Style,NaN,76561198055054795,NaN,CounterTerrorist,NaN,0,...,NaN,HE,0,NaN,-855.077,438.69090,-459.0147,-543.8581,NaN,NaN
4,esea_match_13770997.dem,2,191.0587,Hentai Hooligans,NaN,76561198037331400,NaN,Terrorist,NaN,0,...,NaN,Molotov,0,NaN,-2617.490,-1832.40700,-2743.5610,-927.2995,NaN,NaN


In [13]:
master_grenades.dtypes

file                object
round                int64
seconds            float64
att_team            object
vic_team            object
att_id               int64
vic_id             float64
att_side            object
vic_side            object
hp_dmg               int64
arm_dmg              int64
is_bomb_planted       bool
bomb_site           object
hitbox              object
nade                object
att_rank             int64
vic_rank           float64
att_pos_x          float64
att_pos_y          float64
nade_land_x        float64
nade_land_y        float64
vic_pos_x          float64
vic_pos_y          float64
dtype: object

In [14]:
master_grenades.isnull().head()

,file,round,seconds,att_team,vic_team,att_id,vic_id,att_side,vic_side,hp_dmg,...,hitbox,nade,att_rank,vic_rank,att_pos_x,att_pos_y,nade_land_x,nade_land_y,vic_pos_x,vic_pos_y
0,False,False,False,False,True,False,True,False,True,False,...,True,False,False,True,False,False,False,False,True,True
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,True,False,True,False,True,False,...,True,False,False,True,False,False,False,False,True,True
3,False,False,False,False,True,False,True,False,True,False,...,True,False,False,True,False,False,False,False,True,True
4,False,False,False,False,True,False,True,False,True,False,...,True,False,False,True,False,False,False,False,True,True


In [19]:
master_grenades['vic_id'].isnull().sum() / len(master_grenades)

0.8291492478411805

In [22]:
master_grenades['hitbox'].unique()

array([nan, 'Generic'], dtype=object)

In [23]:
master_grenades['bomb_site'] = master_grenades['bomb_site'].fillna(0)

In [33]:
master_grenades = master_grenades.drop(master_grenades[master_grenades['seconds'].isnull()].index).reset_index(drop=True)

In [34]:
master_grenades.isnull().apply(sum)

file                     0
round                    0
seconds                  0
att_team                 0
vic_team           2478925
att_id                   0
vic_id             2478925
att_side                 0
vic_side           2478925
hp_dmg                   0
arm_dmg                  0
is_bomb_planted          0
bomb_site                0
hitbox             2478925
nade                     0
att_rank                 0
vic_rank           2478925
att_pos_x                0
att_pos_y                0
nade_land_x              0
nade_land_y              0
vic_pos_x          2478925
vic_pos_y          2478925
dtype: int64

In [39]:
master_grenades_miss = master_grenades[master_grenades['vic_id'].isnull()].reset_index(drop=True)

In [41]:
master_grenades_miss

,file,round,seconds,att_team,vic_team,att_id,vic_id,att_side,vic_side,hp_dmg,...,hitbox,nade,att_rank,vic_rank,att_pos_x,att_pos_y,nade_land_x,nade_land_y,vic_pos_x,vic_pos_y
0,esea_match_13770997.dem,1,153.1602,Animal Style,NaN,76561198165334141,NaN,CounterTerrorist,NaN,0,...,NaN,Smoke,0,NaN,-1618.14600,-66.002590,-949.85690,-340.30190,NaN,NaN
1,esea_match_13770997.dem,2,186.8617,Animal Style,NaN,76561198055191021,NaN,CounterTerrorist,NaN,0,...,NaN,HE,0,NaN,-1036.35200,492.167600,-466.86760,-356.96410,NaN,NaN
2,esea_match_13770997.dem,2,187.1122,Animal Style,NaN,76561198055054795,NaN,CounterTerrorist,NaN,0,...,NaN,HE,0,NaN,-855.07700,438.690900,-459.01470,-543.85810,NaN,NaN
3,esea_match_13770997.dem,2,191.0587,Hentai Hooligans,NaN,76561198037331400,NaN,Terrorist,NaN,0,...,NaN,Molotov,0,NaN,-2617.49000,-1832.407000,-2743.56100,-927.29950,NaN,NaN
4,esea_match_13770997.dem,2,191.4502,Animal Style,NaN,76561198055054795,NaN,CounterTerrorist,NaN,0,...,NaN,Smoke,0,NaN,-854.77080,427.486500,-490.52820,-652.89840,NaN,NaN
5,esea_match_13770997.dem,2,191.6225,Animal Style,NaN,76561198165334141,NaN,CounterTerrorist,NaN,0,...,NaN,Flash,0,NaN,-1948.91000,-1184.198000,-1780.65800,-976.40300,NaN,NaN
6,esea_match_13770997.dem,2,204.7617,Hentai Hooligans,NaN,76561197961009213,NaN,Terrorist,NaN,0,...,NaN,HE,0,NaN,-1182.09100,-931.697900,-1726.52800,-428.84770,NaN,NaN
7,esea_match_13770997.dem,2,225.5276,Hentai Hooligans,NaN,76561198133908390,NaN,Terrorist,NaN,0,...,NaN,Smoke,0,NaN,-1436.78900,-720.219500,-1122.89800,-510.92580,NaN,NaN
8,esea_match_13770997.dem,3,304.3628,Hentai Hooligans,NaN,76561198037331400,NaN,Terrorist,NaN,0,...,NaN,HE,0,NaN,-1734.93800,-2341.185000,-2860.17400,-1505.67200,NaN,NaN
9,esea_match_13770997.dem,3,306.8371,Hentai Hooligans,NaN,76561198037331400,NaN,Terrorist,NaN,0,...,NaN,Flash,0,NaN,-2115.31400,-1922.654000,-2684.46800,-1329.79300,NaN,NaN


In [44]:
master_grenades_hit = master_grenades.dropna().reset_index(drop=True)

In [43]:
master_grenades_hit.isnull().apply(sum)

file               0
round              0
seconds            0
att_team           0
vic_team           0
att_id             0
vic_id             0
att_side           0
vic_side           0
hp_dmg             0
arm_dmg            0
is_bomb_planted    0
bomb_site          0
hitbox             0
nade               0
att_rank           0
vic_rank           0
att_pos_x          0
att_pos_y          0
nade_land_x        0
nade_land_y        0
vic_pos_x          0
vic_pos_y          0
dtype: int64

In [45]:
master_grenades_hit['att_side'].unique()#['att_team'].nunique()

array(['Terrorist', 'CounterTerrorist'], dtype=object)

In [46]:
master_grenades_hit['vic_side'].unique()#['att_team'].nunique()

array(['CounterTerrorist', 'Terrorist'], dtype=object)

In [49]:
att = pd.get_dummies(master_grenades_hit['att_side'])
att.head()

,CounterTerrorist,Terrorist
0,0,1
1,1,0
2,1,0
3,1,0
4,1,0


In [50]:
vic = pd.get_dummies(master_grenades_hit['vic_side'])
vic.head()

,CounterTerrorist,Terrorist
0,1,0
1,0,1
2,0,1
3,0,1
4,0,1


In [51]:
X = vic.join(att, lsuffix='_vic', rsuffix='_att')

In [54]:
round_num = pd.get_dummies(master_grenades_hit['round'])

In [55]:
round_num.columns = ['round_' +str(i) for i in round_num.columns]
round_num.head()

,round_1,round_2,round_3,round_4,round_5,round_6,round_7,round_8,round_9,round_10,...,round_54,round_55,round_56,round_57,round_58,round_59,round_60,round_61,round_62,round_63
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
X = X.join(round_num)

In [57]:
X = X.rename(columns = {'None':'None_att'})

In [58]:
X.head()

,CounterTerrorist_vic,Terrorist_vic,CounterTerrorist_att,Terrorist_att,round_1,round_2,round_3,round_4,round_5,round_6,...,round_54,round_55,round_56,round_57,round_58,round_59,round_60,round_61,round_62,round_63
0,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
X['hp_dmg_norm'] = master_grenades_hit.groupby('file')['hp_dmg'].transform(lambda x: (x - x.mean()) / x.std())
X['arm_dmg_norm'] = master_grenades_hit.groupby('file')['arm_dmg'].transform(lambda x: (x - x.mean()) / x.std())
X['seconds_norm'] = master_grenades_hit.groupby('file')['seconds'].transform(lambda x: (x - x.mean()) / x.std())

In [63]:
bomb_planted = pd.get_dummies(master_grenades_hit['is_bomb_planted'])
bomb_planted.columns = ['bomb_planted_False', 'bomb_planted_True']

In [64]:
X = X.join(bomb_planted)

In [65]:
bomb_site = pd.get_dummies(master_grenades_hit['bomb_site'])

In [66]:
bomb_site = bomb_site.rename(columns= {0:'Bomb_Not_Planted', 'A':'Planted_at_A', 'B':'Planted_at_B'})
bomb_site.head()

,Bomb_Not_Planted,Planted_at_A,Planted_at_B
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [67]:
X = X.join(bomb_site)

In [72]:
master_grenades_hit.columns

Index(['file', 'round', 'seconds', 'att_team', 'vic_team', 'att_id', 'vic_id',
       'att_side', 'vic_side', 'hp_dmg', 'arm_dmg', 'is_bomb_planted',
       'bomb_site', 'hitbox', 'nade', 'att_rank', 'vic_rank', 'att_pos_x',
       'att_pos_y', 'nade_land_x', 'nade_land_y', 'vic_pos_x', 'vic_pos_y'],
      dtype='object')

In [80]:
nade = pd.get_dummies(master_grenades_hit['nade'])
nade.head()

,Decoy,Flash,HE,Incendiary,Molotov,Smoke
0,0,0,1,0,0,0
1,0,0,0,1,0,0
2,0,0,0,1,0,0
3,0,0,0,1,0,0
4,0,0,0,1,0,0


In [84]:
X = X.join(nade)

Now we will join master_meta to master_grenades_hit based on 'file' and 'round'

In [85]:
master_meta.head()

,file,map,round,start_seconds,end_seconds,winner_team,winner_side,round_type,ct_eq_val,t_eq_val
0,esea_match_13770997.dem,de_overpass,1,94.30782,160.9591,Hentai Hooligans,Terrorist,PISTOL_ROUND,4300,4250
1,esea_match_13770997.dem,de_overpass,2,160.95910,279.3998,Hentai Hooligans,Terrorist,ECO,6300,19400
2,esea_match_13770997.dem,de_overpass,3,279.39980,341.0084,Hentai Hooligans,Terrorist,SEMI_ECO,7650,19250
3,esea_match_13770997.dem,de_overpass,4,341.00840,435.4259,Hentai Hooligans,Terrorist,NORMAL,24900,23400
4,esea_match_13770997.dem,de_overpass,5,435.42590,484.2398,Animal Style,CounterTerrorist,ECO,5400,20550


In [86]:
master_grenades_hit.head()

,file,round,seconds,att_team,vic_team,att_id,vic_id,att_side,vic_side,hp_dmg,...,hitbox,nade,att_rank,vic_rank,att_pos_x,att_pos_y,nade_land_x,nade_land_y,vic_pos_x,vic_pos_y
0,esea_match_13770997.dem,2,184.7945,Hentai Hooligans,Animal Style,76561198037331400,7.656120e+16,Terrorist,CounterTerrorist,70,...,Generic,HE,0,0.0,-1719.904,-2357.647,-2774.665,-1603.943,-2741.250,-1523.163
1,esea_match_13770997.dem,4,363.9355,Animal Style,Hentai Hooligans,76561198048742997,7.656120e+16,CounterTerrorist,Terrorist,1,...,Generic,Incendiary,0,0.0,-2339.552,-358.022,-1823.104,-2271.759,-1872.836,-2442.808
2,esea_match_13770997.dem,4,363.9355,Animal Style,Hentai Hooligans,76561198048742997,7.656120e+16,CounterTerrorist,Terrorist,1,...,Generic,Incendiary,0,0.0,-2339.552,-358.022,-1823.104,-2271.759,-1713.419,-2356.086
3,esea_match_13770997.dem,4,363.9355,Animal Style,Hentai Hooligans,76561198048742997,7.656120e+16,CounterTerrorist,Terrorist,2,...,Generic,Incendiary,0,0.0,-2339.552,-358.022,-1823.104,-2271.759,-1748.881,-2336.291
4,esea_match_13770997.dem,4,363.9355,Animal Style,Hentai Hooligans,76561198048742997,7.656120e+16,CounterTerrorist,Terrorist,2,...,Generic,Incendiary,0,0.0,-2339.552,-358.022,-1823.104,-2271.759,-1677.455,-2380.632


In [88]:
master_join = master_grenades_hit.merge(master_meta, on=['file', 'round'], how='outer')

In [89]:
len(master_join)

590364

In [92]:
master_join

,file,round,seconds,att_team,vic_team,att_id,vic_id,att_side,vic_side,hp_dmg,...,vic_pos_x,vic_pos_y,map,start_seconds,end_seconds,winner_team,winner_side,round_type,ct_eq_val,t_eq_val
0,esea_match_13770997.dem,2,184.7945,Hentai Hooligans,Animal Style,7.656120e+16,7.656120e+16,Terrorist,CounterTerrorist,70.0,...,-2741.2500,-1523.16300,de_overpass,160.9591,279.3998,Hentai Hooligans,Terrorist,ECO,6300,19400
1,esea_match_13770997.dem,4,363.9355,Animal Style,Hentai Hooligans,7.656120e+16,7.656120e+16,CounterTerrorist,Terrorist,1.0,...,-1872.8360,-2442.80800,de_overpass,341.0084,435.4259,Hentai Hooligans,Terrorist,NORMAL,24900,23400
2,esea_match_13770997.dem,4,363.9355,Animal Style,Hentai Hooligans,7.656120e+16,7.656120e+16,CounterTerrorist,Terrorist,1.0,...,-1713.4190,-2356.08600,de_overpass,341.0084,435.4259,Hentai Hooligans,Terrorist,NORMAL,24900,23400
3,esea_match_13770997.dem,4,363.9355,Animal Style,Hentai Hooligans,7.656120e+16,7.656120e+16,CounterTerrorist,Terrorist,2.0,...,-1748.8810,-2336.29100,de_overpass,341.0084,435.4259,Hentai Hooligans,Terrorist,NORMAL,24900,23400
4,esea_match_13770997.dem,4,363.9355,Animal Style,Hentai Hooligans,7.656120e+16,7.656120e+16,CounterTerrorist,Terrorist,2.0,...,-1677.4550,-2380.63200,de_overpass,341.0084,435.4259,Hentai Hooligans,Terrorist,NORMAL,24900,23400
5,esea_match_13770997.dem,4,363.9355,Animal Style,Hentai Hooligans,7.656120e+16,7.656120e+16,CounterTerrorist,Terrorist,3.0,...,-1773.3850,-2305.94800,de_overpass,341.0084,435.4259,Hentai Hooligans,Terrorist,NORMAL,24900,23400
6,esea_match_13770997.dem,4,363.9355,Animal Style,Hentai Hooligans,7.656120e+16,7.656120e+16,CounterTerrorist,Terrorist,3.0,...,-1674.0680,-2380.28500,de_overpass,341.0084,435.4259,Hentai Hooligans,Terrorist,NORMAL,24900,23400
7,esea_match_13770997.dem,4,363.9355,Animal Style,Hentai Hooligans,7.656120e+16,7.656120e+16,CounterTerrorist,Terrorist,4.0,...,-1788.3610,-2264.08400,de_overpass,341.0084,435.4259,Hentai Hooligans,Terrorist,NORMAL,24900,23400
8,esea_match_13770997.dem,4,363.9355,Animal Style,Hentai Hooligans,7.656120e+16,7.656120e+16,CounterTerrorist,Terrorist,4.0,...,-1802.6200,-2235.25300,de_overpass,341.0084,435.4259,Hentai Hooligans,Terrorist,NORMAL,24900,23400
9,esea_match_13770997.dem,4,363.9355,Animal Style,Hentai Hooligans,7.656120e+16,7.656120e+16,CounterTerrorist,Terrorist,5.0,...,-1810.7760,-2228.35300,de_overpass,341.0084,435.4259,Hentai Hooligans,Terrorist,NORMAL,24900,23400


In [91]:
master_join.isnull().sum()

file                   0
round                  0
seconds            79510
att_team           79510
vic_team           79510
att_id             79510
vic_id             79510
att_side           79510
vic_side           79510
hp_dmg             79510
arm_dmg            79510
is_bomb_planted    79510
bomb_site          79510
hitbox             79510
nade               79510
att_rank           79510
vic_rank           79510
att_pos_x          79510
att_pos_y          79510
nade_land_x        79510
nade_land_y        79510
vic_pos_x          79510
vic_pos_y          79510
map                    0
start_seconds         20
end_seconds           20
winner_team            9
winner_side            0
round_type             0
ct_eq_val              0
t_eq_val               0
dtype: int64

In [90]:
len(master_join[master_join['start_seconds'].isnull()]) / len(master_join)

3.3877404448780755e-05

In [93]:
master_join = master_join[master_join['start_seconds'].isnull()==False]

In [95]:
len(master_join)

590344

In [96]:
len(X) 

510854

In [97]:
len(master_join)

590344

In [98]:
master_join = master_join.reset_index(drop=True)
X = X.reset_index(drop=True)

In [99]:
X = X[master_join['winner_team'].isnull() == False]

/Users/jordanlevy/Desktop/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [100]:
len(X)

510845

In [101]:
X.head()

,CounterTerrorist_vic,Terrorist_vic,CounterTerrorist_att,Terrorist_att,round_1,round_2,round_3,round_4,round_5,round_6,...,bomb_planted_True,Bomb_Not_Planted,Planted_at_A,Planted_at_B,Decoy,Flash,HE,Incendiary,Molotov,Smoke
0,1,0,0,1,0,1,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
1,0,1,1,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,1,0,0
2,0,1,1,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,1,0,0
3,0,1,1,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,1,0,0
4,0,1,1,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,1,0,0


In [102]:
X.columns

Index(['CounterTerrorist_vic', 'Terrorist_vic', 'CounterTerrorist_att',
       'Terrorist_att', 'round_1', 'round_2', 'round_3', 'round_4', 'round_5',
       'round_6', 'round_7', 'round_8', 'round_9', 'round_10', 'round_11',
       'round_12', 'round_13', 'round_14', 'round_15', 'round_16', 'round_17',
       'round_18', 'round_19', 'round_20', 'round_21', 'round_22', 'round_23',
       'round_24', 'round_25', 'round_26', 'round_27', 'round_28', 'round_29',
       'round_30', 'round_31', 'round_32', 'round_33', 'round_34', 'round_35',
       'round_36', 'round_37', 'round_38', 'round_39', 'round_40', 'round_41',
       'round_42', 'round_43', 'round_44', 'round_45', 'round_46', 'round_47',
       'round_48', 'round_49', 'round_50', 'round_51', 'round_52', 'round_53',
       'round_54', 'round_55', 'round_56', 'round_57', 'round_58', 'round_59',
       'round_60', 'round_61', 'round_62', 'round_63', 'hp_dmg_norm',
       'arm_dmg_norm', 'seconds_norm', 'bomb_planted_False',
       'bom

In [103]:
X.head()

,CounterTerrorist_vic,Terrorist_vic,CounterTerrorist_att,Terrorist_att,round_1,round_2,round_3,round_4,round_5,round_6,...,bomb_planted_True,Bomb_Not_Planted,Planted_at_A,Planted_at_B,Decoy,Flash,HE,Incendiary,Molotov,Smoke
0,1,0,0,1,0,1,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
1,0,1,1,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,1,0,0
2,0,1,1,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,1,0,0
3,0,1,1,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,1,0,0
4,0,1,1,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,1,0,0


In [104]:
len(X) == len(master_join)

False

In [105]:
master_join = master_join.dropna()

In [106]:
len(X) == len(master_join)

True

In [107]:
y = pd.get_dummies(master_join['winner_side'])

In [108]:
len(X) == len(y)

True

In [109]:
len(X)

510845

In [110]:
master_join.head()

,file,round,seconds,att_team,vic_team,att_id,vic_id,att_side,vic_side,hp_dmg,...,vic_pos_x,vic_pos_y,map,start_seconds,end_seconds,winner_team,winner_side,round_type,ct_eq_val,t_eq_val
0,esea_match_13770997.dem,2,184.7945,Hentai Hooligans,Animal Style,7.656120e+16,7.656120e+16,Terrorist,CounterTerrorist,70.0,...,-2741.250,-1523.163,de_overpass,160.9591,279.3998,Hentai Hooligans,Terrorist,ECO,6300,19400
1,esea_match_13770997.dem,4,363.9355,Animal Style,Hentai Hooligans,7.656120e+16,7.656120e+16,CounterTerrorist,Terrorist,1.0,...,-1872.836,-2442.808,de_overpass,341.0084,435.4259,Hentai Hooligans,Terrorist,NORMAL,24900,23400
2,esea_match_13770997.dem,4,363.9355,Animal Style,Hentai Hooligans,7.656120e+16,7.656120e+16,CounterTerrorist,Terrorist,1.0,...,-1713.419,-2356.086,de_overpass,341.0084,435.4259,Hentai Hooligans,Terrorist,NORMAL,24900,23400
3,esea_match_13770997.dem,4,363.9355,Animal Style,Hentai Hooligans,7.656120e+16,7.656120e+16,CounterTerrorist,Terrorist,2.0,...,-1748.881,-2336.291,de_overpass,341.0084,435.4259,Hentai Hooligans,Terrorist,NORMAL,24900,23400
4,esea_match_13770997.dem,4,363.9355,Animal Style,Hentai Hooligans,7.656120e+16,7.656120e+16,CounterTerrorist,Terrorist,2.0,...,-1677.455,-2380.632,de_overpass,341.0084,435.4259,Hentai Hooligans,Terrorist,NORMAL,24900,23400


Now we have our features and labels for esea_master_grenades_demos.part1.csv

In [111]:
maps = pd.get_dummies(master_join['map'])
maps.head()

,de_cache,de_cbble,de_dust2,de_inferno,de_mirage,de_nuke,de_overpass,de_train
0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,1,0


In [112]:
X = X.join(maps)

In [113]:
round_types = pd.get_dummies(master_join['round_type'])
round_types.head()

,ECO,FORCE_BUY,NORMAL,PISTOL_ROUND,SEMI_ECO
0,1,0,0,0,0
1,0,0,1,0,0
2,0,0,1,0,0
3,0,0,1,0,0
4,0,0,1,0,0


In [114]:
X = X.join(round_types)

In [115]:
X['ct_eq_val_norm'] = master_join.groupby('file')['ct_eq_val'].transform(lambda x: (x - x.mean()) / x.std())
X['t_eq_val_norm'] = master_join.groupby('file')['t_eq_val'].transform(lambda x: (x - x.mean()) / x.std())

In [116]:
X

,CounterTerrorist_vic,Terrorist_vic,CounterTerrorist_att,Terrorist_att,round_1,round_2,round_3,round_4,round_5,round_6,...,de_nuke,de_overpass,de_train,ECO,FORCE_BUY,NORMAL,PISTOL_ROUND,SEMI_ECO,ct_eq_val_norm,t_eq_val_norm
0,1,0,0,1,0,1,0,0,0,0,...,0,1,0,1,0,0,0,0,-4.172405,0.480077
1,0,1,1,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,-0.815998,1.063998
2,0,1,1,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,-0.815998,1.063998
3,0,1,1,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,-0.815998,1.063998
4,0,1,1,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,-0.815998,1.063998
5,0,1,1,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,-0.815998,1.063998
6,0,1,1,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,-0.815998,1.063998
7,0,1,1,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,-0.815998,1.063998
8,0,1,1,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,-0.815998,1.063998
9,0,1,1,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,-0.815998,1.063998


In [117]:
X.to_csv('CSGO_Data_Cleaning/esea_master_grenades_hit_meta_cleaned.part1.csv')

In [118]:
X

,CounterTerrorist_vic,Terrorist_vic,CounterTerrorist_att,Terrorist_att,round_1,round_2,round_3,round_4,round_5,round_6,...,de_nuke,de_overpass,de_train,ECO,FORCE_BUY,NORMAL,PISTOL_ROUND,SEMI_ECO,ct_eq_val_norm,t_eq_val_norm
0,1,0,0,1,0,1,0,0,0,0,...,0,1,0,1,0,0,0,0,-4.172405,0.480077
1,0,1,1,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,-0.815998,1.063998
2,0,1,1,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,-0.815998,1.063998
3,0,1,1,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,-0.815998,1.063998
4,0,1,1,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,-0.815998,1.063998
5,0,1,1,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,-0.815998,1.063998
6,0,1,1,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,-0.815998,1.063998
7,0,1,1,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,-0.815998,1.063998
8,0,1,1,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,-0.815998,1.063998
9,0,1,1,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,-0.815998,1.063998
